In [0]:
dbutils.fs.ls(f"abfss://logs-raw@{storage_account_name}.dfs.core.windows.net/")


[FileInfo(path='abfss://logs-raw@sfhexastorage.dfs.core.windows.net/raw.json', name='raw.json', size=138, modificationTime=1760610959000)]

In [0]:
# Install Snowflake Connector and Snowpark packages
%pip install snowflake-snowpark-python
%pip install snowflake-connector-python
%pip install snowflake-sqlalchemy


In [0]:
%restart_python 

In [0]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, to_timestamp

# -----------------------------
# Snowflake Connection
# -----------------------------
sfOptions = {
    "account": "sqishot-fa68768", 
    "user": "poojashree",
    "password": "Poojashree@307",
    "warehouse": "COMPUTE_WH",
    "database": "LOG_ANALYTICS",
    "schema": "PUBLIC",
}

session = Session.builder.configs(sfOptions).create()
print("Connected to Snowflake successfully!")

# -----------------------------
# Connect to ADLS using Storage Account Key
# -----------------------------
storage_account_name = "sfhexastorage"
storage_account_key = "jBmM0OVBS0I8tvj1LLFb0qzJVIFk9+Qd/hNW0ZvNe6TiSbatHvHC6NoeSYm3nfbmy1hocQuVbnhw+AStRqeGIA=="

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

# -----------------------------
# Read JSON logs from ADLS raw folder
# -----------------------------
logs_path = f"abfss://logs-raw@{storage_account_name}.dfs.core.windows.net/raw.json"

logs_df = spark.read.json(logs_path)
logs_df.show()  # Quick preview

# Convert Spark DataFrame to Pandas (for small batch)
logs_pd = logs_df.toPandas()

# -----------------------------
# Create Snowpark DataFrame from Pandas
# -----------------------------
sp_df = session.create_dataframe(logs_pd)

# -----------------------------
# Rename + Format Columns
# -----------------------------
sp_df_cleaned = (
    sp_df
    .with_column_renamed("user", "USER_NAME")
    .with_column_renamed("event", "EVENT")
    .with_column("EVENT_TIME", to_timestamp(col('"timestamp"')))
)

sp_df_cleaned.show()
'''
# -----------------------------
# Create database/schema/table if not exists
# -----------------------------
session.sql("CREATE DATABASE IF NOT EXISTS LOG_ANALYTICS").collect()
session.sql("USE DATABASE LOG_ANALYTICS").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS PUBLIC").collect()
session.sql("""
    CREATE OR REPLACE TABLE LOGS_CLEANED (
        EVENT STRING,
        USER_NAME STRING,
        TIMESTAMP STRING,
        EVENT_TIME TIMESTAMP_NTZ
    )
""").collect()
'''
# -----------------------------
# Write cleaned data to Snowflake table
# -----------------------------
# Automatic insert, append mode
sp_df_cleaned.write.mode("append").option("column_quote", '"').save_as_table("LOGS_CLEANED")
print("Data written to Snowflake table LOGS_CLEANED successfully!")


Connected to Snowflake successfully!
+------+--------------------+-----+
| event|           timestamp| user|
+------+--------------------+-----+
| login|2025-10-16T09:23:00Z|pooja|
|logout|2025-10-16T09:30:00Z| ravi|
+------+--------------------+-----+

-----------------------------------------------------------------
|"event"  |"timestamp"           |"user"  |"EVENT_TIME"         |
-----------------------------------------------------------------
|login    |2025-10-16T09:23:00Z  |pooja   |2025-10-16 09:23:00  |
|logout   |2025-10-16T09:30:00Z  |ravi    |2025-10-16 09:30:00  |
-----------------------------------------------------------------

Data written to Snowflake table LOGS_CLEANED successfully!
